### 3.4 Clustering

Python program to evaluate performance using clustering on the datasets obtained in Task 3.1.

#### Markdown Question: 
**Question**: Between clustering and classification algorithms, which one performed better for star dataset. Justify your answer.